In [2]:
#Análise Exploratória do Dataset

In [3]:
import pandas as pd

In [4]:
dados_treino = pd.read_csv("train.csv")
dados_teste = pd.read_csv("test.csv")

In [5]:
dados_treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
dados_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
dados_treino.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
dados_teste.drop(labels=['PassengerId','Name', 'Ticket', 'Cabin'], axis=1, inplace = True)
dados_treino.drop(labels=['PassengerId','Name', 'Ticket', 'Cabin'], axis=1, inplace = True)

In [9]:
#Transformando em Variáveis Categóricas em Várias Dummy através do pd.getdummies

In [10]:
pd.get_dummies?

In [11]:
dados_treino = pd.get_dummies(dados_treino, columns=["Sex"])
dados_treino = pd.get_dummies(dados_treino, columns=["Embarked"])
dados_teste = pd.get_dummies(dados_teste, columns=["Sex"])
dados_teste = pd.get_dummies(dados_teste, columns=["Embarked"])

In [12]:
# Definindo as variáveis preditoras e a variável target para o modelo

In [13]:
X = dados_treino.loc[:, dados_treino.columns!= "Survived"]
y = dados_treino["Survived"]

In [14]:
# Inserindo dados para os Valores Ausentes atráves do Método dos Vizinhos Próximos (KNN)

In [15]:
from sklearn.impute import KNNImputer

In [16]:
imputar = KNNImputer()

In [17]:
imputar.fit(X)

KNNImputer()

In [18]:
dados_teste = imputar.transform(dados_teste)

In [19]:
X = imputar.transform(X)

In [20]:
# Construindo o modelo preditivo através do algoritmo "Árvore de Decisão"

In [21]:
from sklearn.tree import DecisionTreeClassifier

In [22]:
modelo = DecisionTreeClassifier(criterion="entropy", max_depth = 6 , min_samples_split=5)

In [23]:
modelo.fit(X,y)

DecisionTreeClassifier(criterion='entropy', max_depth=6, min_samples_split=5)

In [24]:
#Verificando a acuracia do modelo atráves da Validação Cruzada

In [25]:
from sklearn.model_selection import cross_val_score

In [26]:
cross_val_score(modelo, X, y, cv = 10).mean()

0.8216354556803995

In [27]:
# Exportando o resultado das previsões utilizando os Dados de Teste

In [28]:
ID_dados_teste = pd.read_csv("test.csv")

In [29]:
previsao = modelo.predict(dados_teste)

In [30]:
df_previsao = pd.DataFrame(previsao, columns= ['Survived'])
df_previsao['PassengerId'] = ID_dados_teste['PassengerId']

In [31]:
df_previsao = df_previsao[['PassengerId', 'Survived']]

In [32]:
df_previsao.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [34]:
df_previsao.to_csv('submissao.csv', index=False)